In [ ]:
#calculate collaboration score
import pandas as pd
import math
address = "results/flink/file_review_num.csv"
numresult_df = pd.read_csv(address)
address = "results/flink/file_review_num.csv"
timeresult_df = pd.read_csv(address)
address = "results/flink/file_review_num.csv"
sqrtresult_df = pd.read_csv(address)

for col in numresult_df.columns[1:]:
    numresult_df[col].values[:] = 0
for col in timeresult_df.columns[1:]:
    timeresult_df[col].values[:] = 0
for col in sqrtresult_df.columns[1:]:
    sqrtresult_df[col].values[:] = 0

address = "D:/Master/semester3/experiments/dataset/csv/github/apache/flink/2020-01-10/review.csv"
review_df = pd.read_csv(address)
address = "D:/Master/semester3/experiments/features/flink/datedpullauthors.csv"
author_df = pd.read_csv(address)
for i in range(numresult_df.shape[0]):
    number = numresult_df.loc[i, 'number']
    myauthor_df = author_df[author_df.pull_number == number]
    if myauthor_df.shape[0] == 0:
        continue
    author = myauthor_df.iloc[0, 1]
    date = myauthor_df.iloc[0, 2]
    print(i)
    print(number)
    pull_df = author_df[author_df.date < date]
    pull_df = author_df[author_df.author_login == author]
    for j in range(pull_df.shape[0]):
        myreview_df = review_df[review_df.date < date]
        pull_number = pull_df.iloc[j, 0]
        newreview_df = myreview_df[myreview_df.pull_number == pull_number]
        newreview_df = newreview_df.drop_duplicates(subset=['pull_number', 'reviewer_login'], keep="last")
        for k in range(newreview_df.shape[0]):
            reviewer = newreview_df.iloc[k, 1]
            newdate = newreview_df.iloc[k, 2]
            numresult_df.loc[i, reviewer] += 1
            lasttime = time_interval(newdate, date)
            if lasttime == 0:
                lasttime = 1
            if lasttime < timeresult_df.loc[i, reviewer] or timeresult_df.loc[i, reviewer] == 0:
                timeresult_df.loc[i, reviewer] = lasttime
for i in range(sqrtresult_df.shape[0]):
    for j in range(1, sqrtresult_df.shape[1]):
        if timeresult_df.iloc[i, j] ==0:
            timeresult_df.iloc[i, j] = 1
        sqrtresult_df.iloc[i, j] = numresult_df.iloc[i, j]/math.sqrt(timeresult_df.iloc[i, j])
        timeresult_df.iloc[i, j] = numresult_df.iloc[i, j]/timeresult_df.iloc[i, j]
        
numresult_df.to_csv("results/flink/collaboration_num.csv", index = False)
timeresult_df.to_csv("results/flink/collaboration_time.csv", index = False)
sqrtresult_df.to_csv("results/flink/collaboration_sqrt.csv", index = False)